In [2]:
import pandas as pd
import numpy as np
import requests
import json
from io import StringIO

In [3]:
# The URL from which to download the CSV
URL = "https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?fourfour=43nn-pn8j&cacheBust=1712428502&date=20240407&accessType=DOWNLOAD"

# Send a GET request to the URL
response = requests.get(URL)
# Check if the request was successful
if response.status_code == 200:
    # Open a file in binary write mode
    with open('nyc_restaurants_data.csv', 'wb') as file:
        # Write the content of the response to the file
        file.write(response.content)
    print("CSV file has been downloaded and saved as 'nyc_restaurants_data.csv'.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

CSV file has been downloaded and saved as 'nyc_restaurants_data.csv'.


In [4]:
import pandas as pd

df = pd.read_csv('nyc_restaurants_data.csv')

In [6]:
#Dataframe for Brooklyn
df_brooklyn = df[df['BORO'] == 'Brooklyn']

df_brooklyn

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50080855,KING OF THE SEA,Brooklyn,1102,BEDFORD AVENUE,11216.0,9147554683,Seafood,10/30/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.687000,-73.954647,303.0,36.0,22900.0,3056465.0,3.019710e+09,BK69,NaN
1,50007717,MY HOUSE,Brooklyn,1650,SHEEPSHEAD BAY ROAD,11235.0,7189340400,American,05/25/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.586538,-73.952382,315.0,48.0,60600.0,3204919.0,3.074590e+09,BK17,NaN
2,40948509,GOLDEN KRUST CARIBBEAN BAKERY & GRILL,Brooklyn,1936,RALPH AVENUE,11236.0,7185312828,Caribbean,08/13/2021,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.630364,-73.918432,318.0,46.0,72000.0,3214956.0,3.077630e+09,BK58,NaN
5,50134862,THE CHOCOLATE FACTORY,Brooklyn,70,SCOTT AVENUE,11237.0,6462440734,NaN,01/01/1900,NaN,...,NaN,40.709853,-73.922706,301.0,34.0,44900.0,3070597.0,3.029900e+09,BK90,NaN
7,50149715,NaN,Brooklyn,1137,NOSTRAND AVENUE,11225.0,3058903940,NaN,01/01/1900,NaN,...,NaN,40.659823,-73.950528,309.0,40.0,80400.0,3106905.0,3.048010e+09,BK60,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226385,50041401,FUSION EAST,Brooklyn,1165,ELTON STREET,11239.0,7189755065,Caribbean,02/23/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.656053,-73.873720,305.0,42.0,107000.0,3398621.0,3.044528e+09,BK82,NaN
226388,50131646,GINJAN CAFE,Brooklyn,333,NOSTRAND AVENUE,11216.0,9292489696,African,08/24/2023,Violations were cited in the following area(s).,...,Pre-permit (Operational) / Initial Inspection,40.686818,-73.950850,303.0,36.0,25100.0,3050881.0,3.018080e+09,BK75,NaN
226392,50145559,TEX*S CHICKEN & BURGERS,Brooklyn,2028,BATH AVENUE,11214.0,7185130420,Chicken,04/22/2024,Violations were cited in the following area(s).,...,Pre-permit (Operational) / Initial Inspection,40.600845,-74.000702,311.0,47.0,28600.0,3168847.0,3.064430e+09,BK27,NaN
226395,50065314,SWEETGREEN,Brooklyn,50,WASHINGTON STREET,11201.0,3479874185,Salads,04/26/2022,Violations were cited in the following area(s).,...,Calorie Posting / Initial Inspection,40.702827,-73.989617,302.0,33.0,2100.0,3329425.0,3.000370e+09,BK38,NaN


In [7]:
df_brooklyn = pd.concat([df_brooklyn], ignore_index=True)

In [8]:
df_brooklyn.shape

(60878, 27)

In [9]:
!pip install google-cloud-storage

In [11]:
#Code for Google Cloud Upload
from google.cloud import storage
from io import StringIO
from io import StringIO
import numpy as np
import pandas as pd
import json

# Specify the name of your bucket
bucket_name = 'restaurant-dataset'

#Specify the path to your JSON configuration file
config_file_path = '/content/nyc-restaurants-419717-5360ff41f4e2.json'

# Authenticate using a service account key
storage_client = storage.Client.from_service_account_json('/content/nyc-restaurants-419717-5360ff41f4e2.json')

# Access the specified bucket
bucket = storage_client.bucket(bucket_name)

# Specify the folder name within the bucket
folder_name = 'df_raw'

#specify name of the CSV file in the bucket
blob = bucket.blob (f'{folder_name}/BrooklynInspect.csv')

# Convert DataFrame to CSV
output = StringIO()
df_brooklyn.to_csv(output, index = False)
data = output.getvalue()
output.close()

try:
    # Upload CSV file to the bucket
    blob.upload_from_string(data, content_type='text/csv')
    print("File uploaded successfully!")
except Exception as e:
    print(f"Error uploading file: {e}")

File uploaded successfully!
